In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os

os.makedirs('/content/object_detection_demo_flow/fine_tuned_model/', exist_ok=True)
os.makedirs('/content/object_detection_demo_flow/training/', exist_ok=True)

!cp -r "/content/gdrive/My Drive/Capstone/Model/20.11.01_sota/fine_tuned_model/" "/content/object_detection_demo_flow/"
!cp -r "/content/gdrive/My Drive/Capstone/Model/20.11.01_sota/training/" "/content/object_detection_demo_flow/"

In [3]:
%%capture
import re
import numpy as np

output_directory = '/content/object_detection_demo_flow/fine_tuned_model'
model_dir = '/content/object_detection_demo_flow/training'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

In [4]:
!ls {output_directory}

checkpoint			model.ckpt.index  saved_model
frozen_inference_graph.pb	model.ckpt.meta
model.ckpt.data-00000-of-00001	pipeline.config


In [5]:
pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)
!ls -alh {pb_fname}

-rw------- 1 root root 19M Nov  1 08:30 /content/object_detection_demo_flow/fine_tuned_model/frozen_inference_graph.pb


In [6]:
os.makedirs('/content/object_detection_demo_flow/data/images/final_test', exist_ok=True)
!cp -r "/content/gdrive/My Drive/Capstone/Sample/" "/content/object_detection_demo_flow/data/images/final_test"

In [ ]:

os.makedirs('/content/object_detection_demo_flow/data/annotations/', exist_ok=True)

label_map_pbtxt_fname = '/content/object_detection_demo_flow/data/annotations/label_map.pbtxt'

In [7]:
import glob

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = pb_fname

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR = '/content/object_detection_demo_flow/data/images/final_test/Sample'

assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)

NameError: ignored